In [1]:
import sys

sys.path.append('./')

In [19]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_df = pd.read_csv('./data/Train.csv')
test_df = pd.read_csv('./data/Test.csv')


In [10]:
train_df['Path']

0        Train/20/00020_00000_00000.png
1        Train/20/00020_00000_00001.png
2        Train/20/00020_00000_00002.png
3        Train/20/00020_00000_00003.png
4        Train/20/00020_00000_00004.png
                      ...              
39204    Train/42/00042_00007_00025.png
39205    Train/42/00042_00007_00026.png
39206    Train/42/00042_00007_00027.png
39207    Train/42/00042_00007_00028.png
39208    Train/42/00042_00007_00029.png
Name: Path, Length: 39209, dtype: object

In [12]:
img_path = os.path.join('./data', train_df.iloc[0]['Path'])
img = Image.open(img_path)
print(img.mode)

RGB


In [13]:
def load_and_preprocess_image(path, img_width, img_height):
    img = Image.open(path)
    img = img.resize((img_width, img_height))
    img = np.array(img) / 255.0
    return img

In [15]:
# Apply this function on all training images
train_images = []
train_labels = []

img_width = 30
img_height = 30

for index, row in train_df.iterrows():
    img_path = os.path.join('./data/', row['Path'])
    train_images.append(load_and_preprocess_image(img_path, img_width, img_height))
    train_labels.append(row['ClassId'])

In [17]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Convert labels to one hot encoding
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [20]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))


Epoch 1/20
617/981 [=================>............] - ETA: 21s - loss: 1.5868 - accuracy: 0.5578

KeyboardInterrupt: 